<a href="https://colab.research.google.com/github/Naveed-Bhutto/PIAIC-Batch-61-AI-201/blob/main/Function_calling_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini API: Function calling config

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

Specifying a function_calling_config allows you to control how the Gemini API acts when tools have been specified. For example, you can choose to only allow free-text output (disabling function calling), force it to choose from a subset of the functions provided in tools, or let it act automatically.

This guide assumes you are already familiar with function calling. For an introduction, check out the docs.

In [1]:
!pip install -U -q "google-generativeai>=0.7.2"

**To** run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [2]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## Set up a model with tools

This example uses 3 functions that control a simple hypothetical lighting system. Using these functions requires them to be called in a specific order. For example, you must turn the light system on before you can change color.

While you can pass these directly to the model and let it try to call them correctly, specifying the `function_calling_config` gives you precise control over the functions that are available to the model.

In [3]:
def enable_lights():
    """Turn on the lighting system."""    # Doc string == """ Description of function, for what purpose function is made"""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


light_controls = [enable_lights, set_light_color, stop_lights]
instruction = """You are a helpful lighting system bot. You can turn lights on and off,
              and you can set the color. Do not perform any other tasks."""     # This is System Function, setting persona for LLM AI model

model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=light_controls,
    system_instruction=instruction
)

chat = model.start_chat()

Create a helper function for setting function_calling_config on tool_config.

In [4]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

## Text-only mode: `NONE`

If you have provided the model with tools, but do not want to use those tools for the current conversational turn, then specify `NONE` as the mode. `NONE` tells the model not to make any function calls, and will behave as though none have been provided.

In [5]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)

I can turn lights on and off, and I can set the color of the lights.



##Automatic mode: AUTO


To allow the model to decide whether to respond in text or call specific functions, you can specify AUTO as the mode.


In [6]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[0])
chat.rewind();  # You are not actually calling the function, so remove this from the history.

function_call {
  name: "enable_lights"
  args {
  }
}



##Function-calling mode: ANY

Setting the mode to ANY will force the model to make a function call. By setting allowed_function_names, the model will only choose from those functions. If it is not set, all of the functions in tools are candidates for function calling.

In this example system, if the lights are already on, then the user can change color or turn the lights off.

In [7]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place PURPLE!", tool_config=tool_config)
print(response.parts[0])

function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "FF00FF"
      }
    }
  }
}



##Automatic function calling

tool_config works when enabling automatic function calling too.

In [8]:
available_fns = ["enable_lights","set_light_color", "stop_lights"]
tool_config = tool_config_from_mode("any", available_fns)

# auto_chat = model.start_chat(enable_automatic_function_calling=True)
# auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)

In [10]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)

LIGHTBOT: Lights enabled.


In [11]:
!pip install -U -q "google-generativeai>=0.7.2"

from google.colab import userdata
import google.generativeai as genai
import google.generativeai as genai
from typing import Literal

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

##Define an API function
Create a function that makes an API request. This function should be defined within the code of your application, but could call services or APIs outside of your application. The Gemini API does not call this function directly, so you can control how and when this function is executed through your application code. For demonstration purposes, this tutorial defines a mock API function that just returns the requested lighting values:



In [12]:
def set_light_values(brightness: int, color_temp: str):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

##Declare functions during model initialization
When you want to use function calling with a model, you must declare your functions when you initialize the model object. You declare functions by setting the model's tools parameter:

In [13]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])



##Generate a function call
Once you have initialized model with your function declarations, you can prompt the model with the defined function. You should use function calling using chat prompting (sendMessage()), since function calling generally benefits from having the context of previous prompts and responses.

The Python SDK's ChatSession object simplifies managing chat sessions by handling the conversation history for you. You can use the enable_automatic_function_calling to have the SDK automatically call the function.

In [14]:
# Create a chat session that automatically makes suggested function calls
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
print(response.text)

OK. I've dimmed the lights to 30% brightness and set the color temperature to warm.  Is there anything else?



##Parallel function calling
In addition to basic function calling described above, you can also call multiple functions in a single turn. This section shows an example for how you can use parallel function calling.

Define the tools.

In [15]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

Now call the model with an instruction that could use all of the specified tools.

In [16]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


Each of the printed results reflects a single function call that the model has requested. To send the results back, include the responses in the same order as they were requested.

In [17]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Party started!  Playing "Never gonna give you up" at 120 bpm.  The disco ball is spinning and the lights are dimmed to 50% brightness.



In [18]:
model._tools.to_proto()

[function_declarations {
   name: "power_disco_ball"
   description: "Powers the spinning disco ball."
   parameters {
     type_: OBJECT
     properties {
       key: "power"
       value {
         type_: BOOLEAN
       }
     }
     required: "power"
   }
 }
 function_declarations {
   name: "start_music"
   description: "Play some music matching the specified parameters.\n\n    Args:\n      energetic: Whether the music is energetic or not.\n      loud: Whether the music is loud or not.\n      bpm: The beats per minute of the music.\n\n    Returns: The name of the song being played.\n    "
   parameters {
     type_: OBJECT
     properties {
       key: "loud"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "energetic"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "bpm"
       value {
         type_: INTEGER
       }
     }
     required: "energetic"
     required: "loud"
     required: "bpm"
   }
 }
 functi

##Function call data type mapping
Automatic schema extraction from Python functions doesn't work in all cases. For example: it doesn't handle cases where you describe the fields of a nested dictionary-object, but the API does support this. The API is able to describe any of the following types:

`AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])`





